In [11]:
import pandas as pd
from pathlib import Path
import re
import numpy as np

import seaborn as sns

In [12]:
# Set BASE_PATH to the directory containing the notebook/script
BASE_PATH = Path().resolve()
REPORTS_PATH = BASE_PATH / 'raw_data'

In [13]:
# Use glob() with the pattern '*.csv' to find all CSV files
csv_files = list(REPORTS_PATH.glob('*.csv'))

In [14]:
# Print the list of found CSV files
for f in csv_files:
    print(f)

C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2019.csv
C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2020.csv
C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2021.csv
C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2022.csv
C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2023.csv
C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2024.csv
C:\Users\Aaron\Coding\epa-aqs-project\assets\reports\raw_data\conreport2025.csv


In [15]:
def extract_year_from_filename(filename):
    """
    Extracts the 4-digit year from a filename like 'conreport2025.csv'.
    Returns the year as a string, or None if not found.
    """
    match = re.search(r'conreport(\d{4})\.csv', str(filename))
    if match:
        return match.group(1)
    return None

def replace_dot_with_nan(df):
    """
    Replace all occurrences of '.' in the DataFrame with np.nan.
    """
    return df.replace('.', np.nan)

In [16]:
all_dataframes = []

for f in csv_files:
    year = extract_year_from_filename(f.name)
    df = pd.read_csv(f)
    
    if year:
        df["Year"] = year
        
    all_dataframes.append(df)

In [17]:
combined_df = pd.concat(all_dataframes, ignore_index=True)

In [18]:
combined_df

,County Code,County,CO 2nd Max 1-hr,CO 2nd Max 8-hr,NO2 98th Percentile 1-hr,NO2 Mean 1-hr,Ozone 2nd Max 1-hr,Ozone 4th Max 8-hr,SO2 99th Percentile 1-hr,SO2 2nd Max 24-hr,SO2 Mean 1-hr,PM2.5 98th Percentile 24-hr,PM2.5 Weighted Mean 24-hr,PM10 2nd Max 24-hr,PM10 Mean 24-hr,Lead Max 3-Mo Avg,Year
0,6001,"Alameda County, CA",5.6,1.7,48,15,0.1,0.072,9,2,0,19,9.4,.,.,.,2019
1,6005,"Amador County, CA",.,.,.,.,0.08,0.067,.,.,.,.,.,.,.,.,2019
2,6007,"Butte County, CA",1.5,1,35,7,0.07,0.063,.,.,.,.,.,53,20,.,2019
3,6009,"Calaveras County, CA",.,.,.,.,0.08,0.066,.,.,.,13,5.5,43,13,.,2019
4,6011,"Colusa County, CA",.,.,.,.,0.06,0.053,.,.,.,24,7,110,28,.,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,6065,"Riverside County, CA",2.3,1.3,.,.,.,.,.,.,.,.,.,.,.,.,2025
309,6067,"Sacramento County, CA",1.3,1.0,.,.,.,.,.,.,.,.,.,.,.,.,2025
310,6071,"San Bernardino County, CA",1.9,1.2,.,.,.,.,.,.,.,.,.,.,.,.,2025
311,6083,"Santa Barbara County, CA",1.1,0.8,.,.,.,.,.,.,.,.,.,.,.,.,2025


In [19]:
combined_df = replace_dot_with_nan(combined_df)

In [20]:
combined_df.to_csv(BASE_PATH / 'combined_conreport.csv', index=False)

In [21]:
pivot = combined_df.pivot_table(
    index=["Year","County Code", "County"],
    values=[
        'CO 2nd Max 1-hr', 'CO 2nd Max 8-hr',
        'NO2 98th Percentile 1-hr', 'NO2 Mean 1-hr', 'Ozone 2nd Max 1-hr',
        'Ozone 4th Max 8-hr', 'SO2 99th Percentile 1-hr', 'SO2 2nd Max 24-hr',
        'SO2 Mean 1-hr', 'PM2.5 98th Percentile 24-hr',
        'PM2.5 Weighted Mean 24-hr', 'PM10 2nd Max 24-hr', 'PM10 Mean 24-hr',
        'Lead Max 3-Mo Avg'
    ],
    aggfunc="mean"  # You can change to 'sum', 'max', etc. if needed
)

In [22]:
pivot.to_csv(BASE_PATH / 'pivot_table_output.csv')